<a href="https://colab.research.google.com/github/mgerlach/advent_of_code/blob/main/2025/aoc2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Utilities

## Read Input

In [2]:
from IPython.core.display import clear_output
import requests

def get_aoc_input(year, day):
    """
    Retrieves the Advent of Code input for a given year and day.

    Args:
        year (int): The Advent of Code year.
        day (int): The Advent of Code day.

    Returns:
        str: The input data as a string, or None if retrieval fails.
    """
    session_cookie = input("Please enter your Advent of Code session cookie: ")
    if not session_cookie:
        print("Session cookie is required to retrieve Advent of Code input.")
        return None

    clear_output();

    headers = {
        'Cookie': f'session={session_cookie}'
    }

    # Advent of Code typically doesn't provide a direct URL for 'test' input.
    # The standard input URL is for the puzzle input.
    # For test inputs, users usually copy-paste from the problem description.
    # This function will retrieve the *puzzle* input. If you need test input
    # you will generally have to manually copy it from the problem page.
    url = f"https://adventofcode.com/{year}/day/{day}/input"

    print(f"Attempting to retrieve Advent of Code {year} Day {day} puzzle input from {url}...")
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status() # Raise an exception for HTTP errors
        return response.text.strip()
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            print(f"Error: Input not found for Advent of Code {year} Day {day}. Check if the year/day is valid or if it's released yet. (Status Code: 404)")
        elif e.response.status_code == 400:
            print(f"Error: Bad request. This often indicates an invalid session cookie. (Status Code: 400)")
        else:
            print(f"HTTP error occurred: {e}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while making the request: {e}")
        return None

## Read Test Input

Attempt to read test input. Needs verification, alternatively copy/paste test input into a multi-line string (important to not have a line break after the opening """!)
```
"""line 0
line 1
"""

In [3]:
import requests
from bs4 import BeautifulSoup

def get_aoc_test_input(year, day):
    """
    Attempts to retrieve the Advent of Code test input for a given year and day.
    Prints all candidates for verification and returns first candidate on problem page.
    If the first candidate is incorrect, do not use it, but copy/paste from the problem page into a multiline string!

    Args:
        year (int): The Advent of Code year.
        day (int): The Advent of Code day.

    Returns:
        str: The input data as a string, or None if retrieval fails.
    """
    session_cookie = input("Please enter your Advent of Code session cookie: ")
    if not session_cookie:
        print("Session cookie is required to retrieve Advent of Code input.")
        return None

    clear_output();

    headers = {
        'Cookie': f'session={session_cookie}'
    }

    # Construct the URL for the problem page
    problem_url = f"https://adventofcode.com/{year}/day/{day}"

    print(f"Attempting to retrieve Advent of Code {year} Day {day} problem page from {problem_url}...")

    try:
        # Make the HTTP GET request
        response = requests.get(problem_url, headers=headers)
        response.raise_for_status() # Raise an exception for HTTP errors (4xx or 5xx)
        problem_page_html = response.text
        print("Successfully retrieved problem page HTML.")
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            print(f"Error: Problem page not found for Advent of Code {year} Day {day}. Check if the year/day is valid. (Status Code: 404)")
        elif e.response.status_code == 400:
            print(f"Error: Bad request. This often indicates an invalid session cookie. (Status Code: 400)")
        else:
            print(f"HTTP error occurred: {e}")
        problem_page_html = None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while making the request: {e}")
        problem_page_html = None

    if problem_page_html:
        print(f"\nFirst 500 characters of retrieved HTML:\n---\n{problem_page_html[:500]}\n---")
    else:
        print(f"Failed to retrieve Advent of Code {year} Day {day} problem page HTML.")
        return None

    soup = BeautifulSoup(problem_page_html, 'html.parser')

    # Find all <code> tags
    code_tags = soup.find_all('code')

    print(f"Found {len(code_tags)} <code> tags. Inspecting content for test input:")
    print("---")

    test_input_candidates = []
    for i, code_tag in enumerate(code_tags):
        # Advent of Code usually puts test inputs in <pre><code>...</code></pre> blocks
        # or sometimes directly in <code> tags within paragraphs.
        # We'll collect all of them and let the user visually identify.
        content = code_tag.get_text(strip=True)
        if content:
            test_input_candidates.append(content)
            print(f"Candidate {i+1}:\n{content}\n")

    if test_input_candidates:
        print("Test input candidates extracted. Please review them to identify the actual test input.")
    else:
        print("No text content found within <code> tags. Test input might be in another tag or format.")
        return None

    test_input = test_input_candidates[0] # Assuming Candidate 1 is the correct test input

    print("Identified Test Input:")
    print("---------------------")
    print(test_input)
    print("---------------------")

    return test_input

## Utilities

In [4]:
from collections import Counter
from functools import reduce
from itertools import groupby, islice, takewhile
import math, re

def vec_add(v1, v2):
  return tuple(sum(i) for i in zip(v1, v2))

def vec_sub(v1, v2):
  return tuple(i[0] - i[1] for i in zip(v1, v2))

dirs45 = [(dy, dx) for dy in [-1, 0, 1] for dx in [-1, 0, 1] if dy != 0 or dx != 0]

dir_up = (-1, 0)
dir_right = (0, 1)
dir_down = (1, 0)
dir_left = (0, -1)

dirs90 = [dir_up, dir_right, dir_down, dir_left]
dirs90_symbols = {
    '^': dir_up,
    '>': dir_right,
    'v': dir_down,
    '<': dir_left
}
turn_right = {dir_up: dir_right, dir_right: dir_down, dir_down: dir_left, dir_left: dir_up}
turn_left = {dir_up: dir_left, dir_left: dir_down, dir_down: dir_right, dir_right: dir_up}

def in_range(p, dimensions):
  y, x = p
  height, width = dimensions
  return y >= 0 and y < height and x >= 0 and x < width

def get_cell(p, grid):
  y, x = p
  return grid[y][x]

# use with itertools islice(limit), islice(start, limit, step), takewhile(predicate, iterate(...))
def iterate(start, func):
  current = start
  while True:
    yield current
    current = func(current)

def read_lines(s):
  return [line.strip() for line in s.splitlines()]

def read_grid(s):
  lines = read_lines(s)
  height = len(lines)
  width = len(lines[0])
  return lines, height, width

def sgn(x):
  if x > 0: return 1
  if x < 0: return -1
  return 0

# Problems

## Day 1

### Read Input

In [ ]:
test_input_1 = get_aoc_test_input(2025, 1)

Attempting to retrieve Advent of Code 2025 Day 1 problem page from https://adventofcode.com/2025/day/1...
Successfully retrieved problem page HTML.

First 500 characters of retrieved HTML:
---
<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 1 - Advent of Code 2025</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?32"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?2" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRan
---
Found 77 <code> tags. Inspecting content for test input:
---
Candidate 1:
0

Candidate 2:
99

Candidate 3:
L

Candidate 4:
R

Candidate 5:
11

Candidate 6:
R8

Candidate 7:
19

Candidate 8:
L19

Candidate 9:
0

Candidate 10:
0

Candidate 11:
99

Candidate 12:
99

Candidate 13:
0

Candidate 14:
5

Cand

In [ ]:
test_input_1 = """L68
L30
R48
L5
R60
L55
L1
L99
R14
L82
"""

In [ ]:
input_1 = get_aoc_input(2025, 1)

Attempting to retrieve Advent of Code 2025 Day 1 puzzle input from https://adventofcode.com/2025/day/1/input...


### Part 1

In [ ]:
def compute_delta(instruction) -> int:
  sign = (1 if instruction[0] == 'R' else -1)
  value = int(instruction[1:])
  return sign * value

deltas = [compute_delta(line) for line in read_lines(input_1)]
print(deltas)
print(len(deltas))

def apply_delta(position, delta):
  return (position + delta) % 100

[9, 19, 34, 38, -47, 30, 36, -35, -25, -10, -6, 38, 16, 43, -37, 48, 23, -19, 22, -17, 1, 13, 26, 11, 31, 42, -22, 41, 21, 3, 24, -22, -28, -6, -21, -37, -17, -25, 43, -21, -4, -28, 16, -49, 46, 37, 14, 42, -12, -29, -36, -64, 67, -12, -96, -93, 38, 96, 70, -43, -27, 29, 43, -95, -72, -5, -73, 73, -3, 83, -88, -69, 86, 20, -97, 70, -77, 75, 40, -95, -10, 50, -85, 84, 16, 74, -74, 43, -85, -58, -95, 95, -44, 44, -68, -32, -97, -4, 50, 32, 19, -29, 829, 41, -15, -26, -83, -77, 60, -74, 34, -91, -22, 53, 898, -98, 49, 51, 99, 70, -69, -24, 24, 5, 95, -48, -38, 66, 61, 33, -74, 762, -750, 90, -802, -10, 190, -80, -85, -15, 738, -54, 16, 81, -90, 38, 76, 65, -76, 71, 35, -85, -54, -261, 868, -3, 663, 72, 22, 37, -59, 524, -12, -12, 69, -47, 974, 579, 84, 276, -35, 350, -50, -36, 36, 7, 11, 89, 87, 6, 23, 93, -16, -43, 33, 635, -16, -86, 52, -575, 74, 26, 10, 96, -6, 33, -505, 84, 98, -59, -831, -39, 21, -2, -606, -45, -90, -63, 47, 80, 985, 29, -237, 41, -3, -38, -44, 27, -83, -89, -65, 27,

In [ ]:
positions = reduce(lambda positions, delta: positions + [apply_delta(positions[-1], delta)], deltas, [50])
print(positions)
ends_on_0 = positions.count(0)
ends_on_0

[50, 59, 78, 12, 50, 3, 33, 69, 34, 9, 99, 93, 31, 47, 90, 53, 1, 24, 5, 27, 10, 11, 24, 50, 61, 92, 34, 12, 53, 74, 77, 1, 79, 51, 45, 24, 87, 70, 45, 88, 67, 63, 35, 51, 2, 48, 85, 99, 41, 29, 0, 64, 0, 67, 55, 59, 66, 4, 0, 70, 27, 0, 29, 72, 77, 5, 0, 27, 0, 97, 80, 92, 23, 9, 29, 32, 2, 25, 0, 40, 45, 35, 85, 0, 84, 0, 74, 0, 43, 58, 0, 5, 0, 56, 0, 32, 0, 3, 99, 49, 81, 0, 71, 0, 41, 26, 0, 17, 40, 0, 26, 60, 69, 47, 0, 98, 0, 49, 0, 99, 69, 0, 76, 0, 5, 0, 52, 14, 80, 41, 74, 0, 62, 12, 2, 0, 90, 80, 0, 15, 0, 38, 84, 0, 81, 91, 29, 5, 70, 94, 65, 0, 15, 61, 0, 68, 65, 28, 0, 22, 59, 0, 24, 12, 0, 69, 22, 96, 75, 59, 35, 0, 50, 0, 64, 0, 7, 18, 7, 94, 0, 23, 16, 0, 57, 90, 25, 9, 23, 75, 0, 74, 0, 10, 6, 0, 33, 28, 12, 10, 51, 20, 81, 2, 0, 94, 49, 59, 96, 43, 23, 8, 37, 0, 41, 38, 0, 56, 83, 0, 11, 46, 73, 87, 76, 44, 88, 0, 26, 23, 0, 91, 0, 83, 0, 45, 54, 0, 80, 17, 0, 29, 0, 52, 85, 88, 44, 0, 23, 45, 31, 39, 16, 0, 91, 47, 70, 0, 79, 0, 27, 39, 63, 0, 94, 54, 13, 1, 8, 60, 

999

### Part 2

In [ ]:
# Iterate through the starting positions and their corresponding deltas
# The number of times a multiple of 100 is crossed (including end position) is the absolute
# difference in the 'hundreds' component of the start and end positions.
total_crosses = sum(abs((start_p + d) // 100 - start_p // 100) for start_p, d in zip(positions[:-1], deltas))

print(f"The dial crosses zero or ends on zero (a multiple of 100) a total of {total_crosses} times.")

The dial crosses zero or ends on zero (a multiple of 100) a total of 6099 times.


## Day 2

### Read Input

In [ ]:
test_input_2 = get_aoc_test_input(2025, 2)

Attempting to retrieve Advent of Code 2025 Day 2 problem page from https://adventofcode.com/2025/day/2...
Successfully retrieved problem page HTML.

First 500 characters of retrieved HTML:
---
<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 2 - Advent of Code 2025</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?32"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?2" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRan
---
Found 62 <code> tags. Inspecting content for test input:
---
Candidate 1:
11-22,95-115,998-1012,1188511880-1188511890,222220-222224,
1698522-1698528,446443-446449,38593856-38593862,565653-565659,
824824821-824824827,2121212118-2121212124

Candidate 2:
,

Candidate 3:
-

Candidate 4:
55

Candidate 5:
5

In [ ]:
test_ranges = [tuple(range_str.split('-')) for range_str in reduce(lambda ranges, line: ranges + line.strip(',').split(','), test_input_2.splitlines(), [])]
print(test_ranges)
test_ids = reduce(lambda ids, r: ids + list(range(int(r[0]), int(r[1])+1)), test_ranges, [])
len(test_ids)

[('11', '22'), ('95', '115'), ('998', '1012'), ('1188511880', '1188511890'), ('222220', '222224'), ('1698522', '1698528'), ('446443', '446449'), ('38593856', '38593862'), ('565653', '565659'), ('824824821', '824824827'), ('2121212118', '2121212124')]


106

In [ ]:
input_2 = get_aoc_input(2025, 2)

Attempting to retrieve Advent of Code 2025 Day 2 puzzle input from https://adventofcode.com/2025/day/2/input...


In [ ]:
ranges = [tuple(range_str.split('-')) for range_str in reduce(lambda ranges, line: ranges + line.strip(',').split(','), input_2.splitlines(), [])]
print(ranges)
ids = reduce(lambda ids, r: ids + list(range(int(r[0]), int(r[1])+1)), ranges, [])
len(ids)

[('989133', '1014784'), ('6948', '9419'), ('13116184', '13153273'), ('4444385428', '4444484883'), ('26218834', '26376188'), ('224020', '287235'), ('2893', '3363'), ('86253', '115248'), ('52', '70'), ('95740856', '95777521'), ('119', '147'), ('67634135', '67733879'), ('2481098640', '2481196758'), ('615473', '638856'), ('39577', '47612'), ('9444', '12729'), ('93', '105'), ('929862406', '930001931'), ('278', '360'), ('452131', '487628'), ('350918', '426256'), ('554', '694'), ('68482544', '68516256'), ('419357748', '419520625'), ('871', '1072'), ('27700', '38891'), ('26', '45'), ('908922', '976419'), ('647064', '746366'), ('9875192107', '9875208883'), ('3320910', '3352143'), ('1', '19'), ('373', '500'), ('4232151', '4423599'), ('1852', '2355'), ('850857', '889946'), ('4943', '6078'), ('74', '92'), ('4050', '4876')]


1590581

### Part 1
In every range (inclusive) find "invalid" ids, consisting of two equal groups of digits, add up such ids!

In [ ]:
test_sum_even = sum(int(id) for id in map(str, test_ids) if len(id) % 2 == 0 and id[:len(id)//2] == id[len(id)//2:])
test_sum_even

1227775554

In [ ]:
sum_even = sum(int(id) for id in map(str, ids) if len(id) % 2 == 0 and id[:len(id)//2] == id[len(id)//2:])
sum_even

18700015741

### Part 2
Extend definition of invalid to arbritary-length group of digits repeated two or more times!


In [ ]:
def invalid_id(id: str, debug:bool = False) -> bool:
  def test(r, chunk_size) -> bool:
    first_chunk = id[:chunk_size]
    rest_chunks = map(lambda chunk_index: id[chunk_index:chunk_index+chunk_size], r[1:])
    # all rest chunks match the first (i.e., there is not one that is not equal)
    all_chunks_equal = not next((True for chunk in rest_chunks
                                 if chunk != first_chunk), False)
    if debug:
      print(id, list(r), chunk_size, all_chunks_equal)
    return all_chunks_equal
  return next(
      (True for r, chunk_size in [(range(0, len(id), chunk_size), chunk_size) for chunk_size in range(1, len(id) // 2 + 1) if len(id) % chunk_size == 0]
       if test(r, chunk_size)),
      False)

In [ ]:
test_sum = sum(int(id) for id in map(str, test_ids) if invalid_id(id, debug=True))
test_sum

11 [0, 1] 1 True
12 [0, 1] 1 False
13 [0, 1] 1 False
14 [0, 1] 1 False
15 [0, 1] 1 False
16 [0, 1] 1 False
17 [0, 1] 1 False
18 [0, 1] 1 False
19 [0, 1] 1 False
20 [0, 1] 1 False
21 [0, 1] 1 False
22 [0, 1] 1 True
95 [0, 1] 1 False
96 [0, 1] 1 False
97 [0, 1] 1 False
98 [0, 1] 1 False
99 [0, 1] 1 True
100 [0, 1, 2] 1 False
101 [0, 1, 2] 1 False
102 [0, 1, 2] 1 False
103 [0, 1, 2] 1 False
104 [0, 1, 2] 1 False
105 [0, 1, 2] 1 False
106 [0, 1, 2] 1 False
107 [0, 1, 2] 1 False
108 [0, 1, 2] 1 False
109 [0, 1, 2] 1 False
110 [0, 1, 2] 1 False
111 [0, 1, 2] 1 True
112 [0, 1, 2] 1 False
113 [0, 1, 2] 1 False
114 [0, 1, 2] 1 False
115 [0, 1, 2] 1 False
998 [0, 1, 2] 1 False
999 [0, 1, 2] 1 True
1000 [0, 1, 2, 3] 1 False
1000 [0, 2] 2 False
1001 [0, 1, 2, 3] 1 False
1001 [0, 2] 2 False
1002 [0, 1, 2, 3] 1 False
1002 [0, 2] 2 False
1003 [0, 1, 2, 3] 1 False
1003 [0, 2] 2 False
1004 [0, 1, 2, 3] 1 False
1004 [0, 2] 2 False
1005 [0, 1, 2, 3] 1 False
1005 [0, 2] 2 False
1006 [0, 1, 2, 3] 1 False
1

4174379265

In [ ]:
sum_all = sum(int(id) for id in map(str, ids) if invalid_id(id))
sum_all

20077272987

## Day 3

### Read Input

In [5]:
test_input_3 = get_aoc_test_input(2025, 3)

Attempting to retrieve Advent of Code 2025 Day 3 problem page from https://adventofcode.com/2025/day/3...
Successfully retrieved problem page HTML.

First 500 characters of retrieved HTML:
---
<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 3 - Advent of Code 2025</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?32"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?2" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRan
---
Found 24 <code> tags. Inspecting content for test input:
---
Candidate 1:
1

Candidate 2:
9

Candidate 3:
987654321111111
811111111111119
234234234234278
818181911112111

Candidate 4:
12345

Candidate 5:
2

Candidate 6:
4

Candidate 7:
24

Candidate 8:
987654321111111

Candidate 9:
98

Candidate 10:
8

In [5]:
test_input_3 = """987654321111111
811111111111119
234234234234278
818181911112111
"""

In [69]:
input_3 = get_aoc_input(2025, 3)

Attempting to retrieve Advent of Code 2025 Day 3 puzzle input from https://adventofcode.com/2025/day/3/input...


### Part 1
Find max two-digit per line and add them up (digits must appear in order but may be spread out)!

In [6]:
def find_2_max_digits(s: str, debug:bool = False) -> str:
  m1 = max(s)
  p1 = s.find(m1)
  r = m1 + max(s[p1+1:]) if p1 < len(s) - 1 else max(s[:p1]) + m1
  if debug:
    print(s, r)
  return r


In [7]:
sum(map(int, map(lambda s: find_2_max_digits(s, True), test_input_3.splitlines())))


987654321111111 98
811111111111119 89
234234234234278 78
818181911112111 92


357

In [59]:
sum(map(int, map(find_2_max_digits, input_3.splitlines())))

17443

### Part 2
Expand to 12 digits ;)

In [115]:
def find_max_digits(s, n, boundary:str = '9', debug:bool = False):
  if n == 0:
    return ''
  m = max((c for c in s if c <= boundary), default=None)  # max char <= boundary
  p = s.find(m) if m else None  # leftmost occurrence
  if p <= len(s) - n:  # enough digits to the right to fill up n digits
    if debug:
      print(f"{s}, n={n}: +'{m}' @ {p}")
    return m + find_max_digits(s[p+1:], n - 1, debug=debug)  # collect n-1 digits to the right, reset boundary to find any digit
  else:
    if debug:
      print(f"{s}, n={n}: -'{m}' @ {p}")
    return find_max_digits(s, n, boundary=str(int(m) - 1), debug=debug)  # try smaller digit in same range n (lower boundary)

In [114]:
sum(map(int, map(lambda s: find_max_digits(s, 12, debug=True), test_input_3.splitlines())))

987654321111111, n=12: +'9' @ 0
87654321111111, n=11: +'8' @ 0
7654321111111, n=10: +'7' @ 0
654321111111, n=9: +'6' @ 0
54321111111, n=8: +'5' @ 0
4321111111, n=7: +'4' @ 0
321111111, n=6: +'3' @ 0
21111111, n=5: +'2' @ 0
1111111, n=4: +'1' @ 0
111111, n=3: +'1' @ 0
11111, n=2: +'1' @ 0
1111, n=1: +'1' @ 0
811111111111119, n=12: -'9' @ 14
811111111111119, n=12: +'8' @ 0
11111111111119, n=11: -'9' @ 13
11111111111119, n=11: +'1' @ 0
1111111111119, n=10: -'9' @ 12
1111111111119, n=10: +'1' @ 0
111111111119, n=9: -'9' @ 11
111111111119, n=9: +'1' @ 0
11111111119, n=8: -'9' @ 10
11111111119, n=8: +'1' @ 0
1111111119, n=7: -'9' @ 9
1111111119, n=7: +'1' @ 0
111111119, n=6: -'9' @ 8
111111119, n=6: +'1' @ 0
11111119, n=5: -'9' @ 7
11111119, n=5: +'1' @ 0
1111119, n=4: -'9' @ 6
1111119, n=4: +'1' @ 0
111119, n=3: -'9' @ 5
111119, n=3: +'1' @ 0
11119, n=2: -'9' @ 4
11119, n=2: +'1' @ 0
1119, n=1: +'9' @ 3
234234234234278, n=12: -'8' @ 14
234234234234278, n=12: -'7' @ 13
234234234234278, n=12:

3121910778619

In [70]:
sum(map(int, map(lambda s: find_max_digits(s, 12), input_3.splitlines())))

172167155440541